In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
game=pd.read_csv("games.csv")
content=pd.read_json("games_metadata.json",lines=True)

In [3]:
game.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


In [4]:
game=game[:10000]

In [5]:
game=game[["app_id","title"]]
game.head()

,app_id,title
0,13500,Prince of Persia: Warrior Within™
1,22364,BRINK: Agents of Change
2,113020,Monaco: What's Yours Is Mine
3,226560,Escape Dead Island
4,249050,Dungeon of the ENDLESS™


In [6]:
game=game.merge(content,on="app_id")
print(game.shape)
game.head()

(10000, 4)


,app_id,title,description,tags
0,13500,Prince of Persia: Warrior Within™,Enter the dark underworld of Prince of Persia ...,"[Action, Adventure, Parkour, Third Person, Gre..."
1,22364,BRINK: Agents of Change,,[Action]
2,113020,Monaco: What's Yours Is Mine,Monaco: What's Yours Is Mine is a single playe...,"[Co-op, Stealth, Indie, Heist, Local Co-Op, St..."
3,226560,Escape Dead Island,Escape Dead Island is a Survival-Mystery adven...,"[Zombies, Adventure, Survival, Action, Third P..."
4,249050,Dungeon of the ENDLESS™,Dungeon of the Endless is a Rogue-Like Dungeon...,"[Roguelike, Strategy, Tower Defense, Pixel Gra..."


In [7]:
game["description"]=game["description"].apply(lambda x:x.split())
game.head()

,app_id,title,description,tags
0,13500,Prince of Persia: Warrior Within™,"[Enter, the, dark, underworld, of, Prince, of,...","[Action, Adventure, Parkour, Third Person, Gre..."
1,22364,BRINK: Agents of Change,[],[Action]
2,113020,Monaco: What's Yours Is Mine,"[Monaco:, What's, Yours, Is, Mine, is, a, sing...","[Co-op, Stealth, Indie, Heist, Local Co-Op, St..."
3,226560,Escape Dead Island,"[Escape, Dead, Island, is, a, Survival-Mystery...","[Zombies, Adventure, Survival, Action, Third P..."
4,249050,Dungeon of the ENDLESS™,"[Dungeon, of, the, Endless, is, a, Rogue-Like,...","[Roguelike, Strategy, Tower Defense, Pixel Gra..."


In [8]:
game["content"]=game["description"]+game["tags"]
new=game.drop(columns=["description","tags"])

In [9]:
new["content"]=new["content"].apply(lambda x:" ".join(x))

In [10]:
new.head()

,app_id,title,content
0,13500,Prince of Persia: Warrior Within™,Enter the dark underworld of Prince of Persia ...
1,22364,BRINK: Agents of Change,Action
2,113020,Monaco: What's Yours Is Mine,Monaco: What's Yours Is Mine is a single playe...
3,226560,Escape Dead Island,Escape Dead Island is a Survival-Mystery adven...
4,249050,Dungeon of the ENDLESS™,Dungeon of the Endless is a Rogue-Like Dungeon...


In [ ]:
print(new.loc[0,"content"])

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
lemmatizer=WordNetLemmatizer()

In [ ]:
print(lemmatizer.lemmatize("criteria",pos="v"))

In [ ]:
def lemmatize(text):
    y=[]
    for w in text.split():
        new_w=lemmatizer.lemmatize(w)
        y.append(new_w)
    return " ".join(y)

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for w in text.split():
        new_w=ps.stem(w)
        y.append(new_w)
    return " ".join(y)

In [ ]:
print(stem("abilities"))#?????wtf

In [ ]:
new["content"]=new["content"].apply(stem)

In [ ]:
new.head()

In [ ]:
new.loc[4,"content"]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tf_idf=tfidf(stop_words="english",max_features=1000)
vector=tf_idf.fit_transform(new["content"]).toarray()
print(len(vector[0]))

In [ ]:
tf_idf.get_feature_names_out()

In [ ]:
sim=cosine_similarity(vector)

In [ ]:
print(sim.shape)
sim

In [ ]:
top_rec=sorted(list(enumerate(sim[0])),key=lambda x:x[1],reverse=True)
top_rec

In [ ]:
def recommend(game):
    index=new[new["title"]==game].index[0]
    top_rec=sorted(list(enumerate(sim[index])),key=lambda x:x[1],reverse=True)
    for candidate in top_rec[1:11]:
        print(new.iloc[candidate[0]].title)

In [ ]:
game_name="Social Club VR : Casino Nights"
print(new[new["title"]==game_name].content)
recommend(game_name)